In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [13]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

In [3]:
# outputs of CT-RAMP model for tour and trip file
household_model_dir = _join(model_outputs_dir, "main")

# input household and person data
person_file = _join(ctramp_dir, 'main\\personData_' + str(iteration) + '.csv')
household_file = _join(ctramp_dir, 'main\\householdData_' + str(iteration) + '.csv')

person = pd.read_csv(person_file)

hh = pd.read_csv(household_file, usecols = ['hh_id', 'taz'])
hh = hh.rename(columns = {'taz': 'home_zone'})

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

# transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")

In [4]:
def skim_core_to_df(skim, core, cols =['orig', 'dest', 'rail_od']):
    skim_df = pd.DataFrame(skim[core])
    skim_df = pd.melt(skim_df.reset_index(), id_vars='index', value_vars=skim_df.columns)
    skim_df['index'] = skim_df['index'] + 1
    skim_df['variable'] = skim_df['variable'] + 1
    skim_df.columns = cols

    return skim_df

In [5]:
def create_rail_od_pairs(transit_demand_dir, transit_skims_dir, period, acc_egg_modes):
    
    #Creates the Rail OD eligible Files
    for per in period:
        print("Period: ",per)

        rail_demand = omx.open_file(_join(transit_demand_dir, "rail_od_v9_trim_" + per.upper() + ".omx"),'w') 
        for acc_egg in acc_egg_modes:
            print("Access Egress Mode: ",acc_egg)
            trn_skm = omx.open_file(_join(transit_skims_dir, "trnskm" + per.lower() +"_" + acc_egg.lower() + ".omx"))
            ivthvy = np.array(trn_skm['IVTHVY'])
            ivtcom = np.array(trn_skm['IVTCOM'])
            ivtrail = ivthvy + ivtcom
            ivtrail[ivtrail > 0] = 1
            #rail_dmn = trn_dmn_acc * ivtrail
            rail_demand[acc_egg] = ivtrail

        rail_demand.close()

In [5]:
create_rail_od_pairs(transit_demand_dir, transit_skims_dir, period, acc_egg_modes)

Period:  am
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  md
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  pm
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ev
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ea
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR


In [6]:
#trip roster
def create_trip_roster(ctramp_dir, transbay_od, geo_cwks, link21_purp_mapping):
    
    
    ind_trip = pd.read_csv(_join(ctramp_dir, 'main\\indivTripData_' + str(iteration) + '.csv'))
    jnt_trip = pd.read_csv(_join(ctramp_dir, 'main\\jointTripData_' + str(iteration) + '.csv'))
    
    jnt_trip['tours'] = 'joint'
    ind_trip['tours'] = 'inm'
    
    ind_drop_columns = ['avAvailable', 'sampleRate', 'taxiWait', 'singleTNCWait', 
                    'sharedTNCWait', 'orig_walk_segment', 'dest_walk_segment',
                    'person_id', 'person_num', 'parking_taz']

    jnt_drop_columns = ['avAvailable', 'sampleRate', 'taxiWait', 'singleTNCWait', 
                    'sharedTNCWait', 'orig_walk_segment', 'dest_walk_segment',
                'parking_taz', 'num_participants']

    ind_trip = ind_trip.drop(columns = ind_drop_columns)
    jnt_trip = jnt_trip.drop(columns = jnt_drop_columns)

    out_tripdata = pd.concat([ind_trip, jnt_trip])
    
    # add transbay_od to final tours
    out_tripdata = pd.merge(out_tripdata, transbay_od, left_on= ['orig_taz', 'dest_taz'], right_on = ['transbay_o', 'transbay_d'], how = 'left')
    out_tripdata['transbay_od'] = out_tripdata['transbay_od'].fillna(0)

    out_tripdata = out_tripdata.drop(columns = ['transbay_o', 'transbay_d'])
    #print(out_tripdata['transbay_od'].value_counts())

    # add geographies to final tours
    out_tripdata = pd.merge(out_tripdata, geo_cwks, left_on = ['orig_taz'], right_on = ['taz'], how = 'left')
    out_tripdata = out_tripdata.rename(columns = {'rdm_zones':'orig_rdm_zones', 
                                                'super_district': 'orig_super_dist',
                                                'county': 'orig_county'})
    del out_tripdata['taz']

    out_tripdata = pd.merge(out_tripdata, geo_cwks, left_on = ['dest_taz'], right_on = ['taz'], how = 'left')
    out_tripdata = out_tripdata.rename(columns = {'rdm_zones':'dest_rdm_zones', 
                                                'super_district': 'dest_super_dist',
                                                'county': 'dest_county'})

    del out_tripdata['taz']

    out_tripdata = pd.merge(out_tripdata, hh, on = 'hh_id', how = 'left')

    # add prioirty population
    out_tripdata = pd.merge(out_tripdata, pp_perc, left_on = ['home_zone'], right_on = ['taz'], how = 'left')
    print("NAs in PP Share:",  out_tripdata['pp_share'].isna().sum())
    # out_tourdata['pp_share'] = out_tourdata['pp_share'].fillna(0)
    del out_tripdata['taz']
    
    #add link21 purpose definitions
    df = out_tripdata.copy()
    df['new_dest_purp'] = df['dest_purpose']
    df['new_orig_purp'] = df['orig_purpose']
    
    # changing the purpose categories for atwork purpose
    df.loc[(df['tour_purpose'] == 'atwork_eat') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'eatout'
    df.loc[(df['tour_purpose'] == 'atwork_eat') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'eatout'

    df.loc[(df['tour_purpose'] == 'atwork_business') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'business'
    df.loc[(df['tour_purpose'] == 'atwork_business') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'business'

    df.loc[(df['tour_purpose'] == 'atwork_maint') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'othmaint'
    df.loc[(df['tour_purpose'] == 'atwork_maint') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'othmaint'
    
    # adding new link21 trip purpose
    df['link21_tour_purp'] = df['tour_purpose'].map(link21_purp_mapping)
    df['link21_orig_purp'] = df['new_orig_purp'].map(link21_purp_mapping)
    df['link21_dest_purp'] = df['new_dest_purp'].map(link21_purp_mapping)

    df['link21_trip_purp'] = df['link21_dest_purp']
    
    # for last trip on tour
    df1 = df.loc[(df['link21_dest_purp'] == 'home')]
    conditions = [
        df1['link21_tour_purp'].eq('work'),
        df1['link21_tour_purp'].eq('school'),
        ~df1['link21_tour_purp'].isin(['work','school'])
    ]

    choices = ['work', 'school', df1['link21_orig_purp']]
    df1['link21_trip_purp'] = np.select(conditions, choices, default=0)
    df2 = df.loc[(df['link21_dest_purp'] != 'home')]
    df2['link21_trip_purp'] = df2['link21_dest_purp']
    df = pd.concat([df1, df2], ignore_index=True)
    
    df1 = df.loc[df['dest_purpose'] == 'atwork']
    conditions = [
        df1['link21_tour_purp'].eq('business'),
        ~df1['link21_tour_purp'].eq('business')
    ]
    choices = ['business', df1['link21_orig_purp']]
    df1['link21_trip_purp'] = np.select(conditions, choices, default=0)
    
    df2 = df.loc[(df['dest_purpose'] != 'atwork')]
    df = pd.concat([df1, df2], ignore_index=True)
    
    df['trips'] = 1
    
    return df

In [7]:
df_trips = create_trip_roster(ctramp_dir, transbay_od, geo_cwks, link21_purp_mapping)

NAs in PP Share: 0


In [8]:
df_trips.columns

Index(['hh_id', 'tour_id', 'stop_id', 'inbound', 'tour_purpose',
       'orig_purpose', 'dest_purpose', 'orig_taz', 'dest_taz', 'depart_hour',
       'trip_mode', 'tour_mode', 'tour_category', 'tours', 'transbay_od',
       'orig_rdm_zones', 'orig_super_dist', 'orig_county', 'dest_rdm_zones',
       'dest_super_dist', 'dest_county', 'home_zone', 'pp_share',
       'new_dest_purp', 'new_orig_purp', 'link21_tour_purp',
       'link21_orig_purp', 'link21_dest_purp', 'link21_trip_purp', 'trips'],
      dtype='object')

In [8]:
df_trn = df_trips.loc[df_trips['trip_mode'].isin([6,7,8])]

In [9]:
df_trn['trips'] = 1

In [10]:
df_trn['Period'] = df_trn['depart_hour'].map(time_period_mapping)
df_trn['Mode'] = df_trn['trip_mode'].map(mode_cat_mapping)

In [15]:
## Filter out only rail inclusive OD pair
#per = 'am'
# filter by time period
#df_trn_pd = df_trn[df_trn['Period'] == 'am']

In [16]:
# read AM skim
#df_od_pr = omx.open_file(_join(transit_demand_dir, "rail_od_v9_trim_" + per.upper() + ".omx"))

In [18]:
#df_od_pr.list_matrices()

['KNR_TRN_WLK', 'PNR_TRN_WLK', 'WLK_TRN_KNR', 'WLK_TRN_PNR', 'WLK_TRN_WLK']

In [11]:
df_temp = []

for period in time_periods:
    print(f'processing - {period}')
    
    df_od_pr = omx.open_file(_join(transit_demand_dir, "rail_od_v9_trim_" + period.upper() + ".omx"))
    df_trn_pd = df_trn[df_trn['Period'] == period]
    
    #walk transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'WALK_TRANSIT']
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_WLK')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_wlk = pd.merge(df_trn_acc, df_rail_od, 
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    
    # PNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'PNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_PNR')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'PNR_TRN_WLK')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_pnr = pd.concat([df_trn_pnr_inb, df_trn_pnr_outbnd], ignore_index=True)
    
    # KNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'KNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_KNR')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')


    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'KNR_TRN_WLK')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_knr = pd.concat([df_trn_knr_inb, df_trn_knr_outbnd], ignore_index=True)
    
    df_trn_rail = pd.concat([df_trn_wlk, df_trn_pnr, df_trn_knr], ignore_index=True)
    df_temp.append(df_trn_rail)

df_trn_rail = pd.concat(df_temp)

processing - am
processing - md
processing - pm
processing - ev
processing - ea


In [12]:
df_trn_rail['link21_trip_purp'].value_counts()

work        39460
social       4899
othmaint     3347
school       3317
othdiscr     2557
shopping     2468
escort       1209
business       26
Name: link21_trip_purp, dtype: int64

In [65]:
df_region_period = df_trn_rail.groupby(['Mode', 'Period', 'link21_trip_purp'])['trips'].sum().reset_index()

df_region_period['Value'] = df_region_period['trips']
df_region_period = df_region_period.rename(columns={'link21_trip_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'A3.1'
df_region_period['Metric_name'] = 'Mode Shares'
df_region_period['Submetric'] = 'A3.1.1'
df_region_period['Description'] = 'Regional linked trips by time period'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = 'Megaregion'
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'Percentage share'
df_region_period['Total_Increment'] = 'Increment'

In [66]:
df_tb = df_trn_rail.groupby(['transbay_od', 'Mode', 'Period', 'link21_trip_purp'])['trips'].sum().reset_index()

df_tb = df_tb.loc[df_tb['transbay_od'] == 1]

df_tb['Value'] = df_tb['trips']/df_tb['trips'].sum()
df_tb = df_tb.rename(columns={'link21_trip_purp': 'Purpose', 'transbay_od' : 'Zone_ID'})
df_tb = df_tb[['Mode', 'Period', 'Purpose', 'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'A3.1'
df_tb['Metric_name'] = 'trips'
df_tb['Submetric'] = 'A3.1.2'
df_tb['Description'] = 'Linked trips by time period in transbay region'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
#df_region_period['Zone_ID'] =
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'Percentage share'
df_tb['Total_Increment'] = 'Increment'

In [69]:
df_cnty = df_trn_rail.groupby(['orig_county', 'dest_county', 'Mode', 
                            'Period', 'link21_trip_purp'])['trips'].sum().reset_index()

df_cnty['Value'] = df_cnty['trips']
df_cnty = df_cnty.rename(columns={
                                  'link21_trip_purp': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'A3.1'
df_cnty['Metric_name'] = 'Mode Shares'
df_cnty['Submetric'] = 'A3.1.3'
df_cnty['Description'] = 'Regional mode share by time period and origin and destination county'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Units'] = 'Percentage share'
df_cnty['Total_Increment'] = 'Increment'

In [71]:
df_rdm = df_trn_rail.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
                           'Period', 'link21_trip_purp'])['trips'].sum().reset_index()

df_rdm['Value'] = df_rdm['trips']
df_rdm = df_rdm.rename(columns={
                                'link21_trip_purp': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'A3.3'
df_rdm['Metric_name'] = 'Mode Shares'
df_rdm['Submetric'] = 'A3.3.4'
df_rdm['Description'] = 'Regional mode share by time period and origin and destination RDM Zones'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Units'] = 'Percentage share'
df_rdm['Total_Increment'] = 'Increment'

In [72]:
df_sd = df_trn_rail.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
                          'Period', 'link21_trip_purp'])['trips'].sum().reset_index()

df_sd['Value'] = df_sd['trips']

df_sd = df_sd.rename(columns={ 
                              'link21_trip_purp': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})
df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'A3.3'
df_sd['Metric_name'] = 'Mode Shares'
df_sd['Submetric'] = 'A3.3.5'
df_sd['Description'] = 'Regional mode share by time period and origin and destination super districts'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Units'] = 'Percentage share'
df_sd['Total_Increment'] = 'Increment'

In [ ]:
#TO DO : Priority Population

In [1]:
#with pd.ExcelWriter(os.path.join(summary_outputs, 'concept-BY15.xlsx'), engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
#    df_region_period.to_excel(writer, sheet_name='A3.1.1', startcol=0, index=False)
#    df_tb.to_excel(writer, sheet_name='A3.1.2', startcol=0, index=False)
#    df_cnty.to_excel(writer, sheet_name='A3.1.3', startcol=0, index=False)
#    df_rdm.to_excel(writer, sheet_name='A3.1.4', startcol=0, index=False)
#    df_sd.to_excel(writer, sheet_name='A3.1.5', startcol=0, index=False)

In [ ]:
## B1.2 and B1.4 Commuting Trips - Work and Non-work trips

In [14]:
commuting_purposes = params['commuting_purpose_mapping']

In [15]:
df_trn_rail['wrk_nwrk_purp'] = df_trn_rail['link21_trip_purp'].map(commuting_purposes)

In [16]:
work_trips = df_trn_rail.loc[df_trn_rail['wrk_nwrk_purp'] == 'work']

In [17]:
df_region_period = work_trips.groupby(['Mode', 'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_region_period['Value'] = df_region_period['trips']
df_region_period = df_region_period.rename(columns={'wrk_nwrk_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'B1.2'
df_region_period['Metric_name'] = 'Commute trips on network'
df_region_period['Submetric'] = 'B1.2.1'
df_region_period['Description'] = 'Number of linked rail trips for commuting purpose(home-based work)'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = 'Megaregion'
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'Trips'
df_region_period['Total_Increment'] = ''

In [18]:
df_tb = work_trips.groupby(['transbay_od', 'Mode', 'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_tb = df_tb.loc[df_tb['transbay_od'] == 1]

df_tb['Value'] = df_tb['trips']/df_tb['trips'].sum()
df_tb = df_tb.rename(columns={'wrk_nwrk_purp': 'Purpose', 'transbay_od' : 'Zone_ID'})
df_tb = df_tb[['Mode', 'Period', 'Purpose', 'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'B1.2'
df_tb['Metric_name'] = 'Commute trips on network'
df_tb['Submetric'] = 'B1.2.2'
df_tb['Description'] = 'Number of linked rail trips for commuting purpose(home-based work)'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
#df_region_period['Zone_ID'] =
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'Trips'
df_tb['Total_Increment'] = ''

In [19]:
df_cnty = work_trips.groupby(['orig_county', 'dest_county', 'Mode', 
                            'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_cnty['Value'] = df_cnty['trips']
df_cnty = df_cnty.rename(columns={
                                  'wrk_nwrk_purp': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'B1.2'
df_cnty['Metric_name'] = 'Commute trips on network'
df_cnty['Submetric'] = 'B1.2.3'
df_cnty['Description'] = 'Number of linked rail trips for commuting purpose(home-based work)'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Units'] = 'Trips'
df_cnty['Total_Increment'] = ''

In [20]:
df_rdm = work_trips.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
                           'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_rdm['Value'] = df_rdm['trips']
df_rdm = df_rdm.rename(columns={
                                'wrk_nwrk_purp': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'B1.2'
df_rdm['Metric_name'] = 'Commute trips on network'
df_rdm['Submetric'] = 'B1.2.4'
df_rdm['Description'] = 'Number of linked rail trips for commuting purpose(home-based work)'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Units'] = 'Trips'
df_rdm['Total_Increment'] = ''

In [21]:
df_sd = work_trips.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
                          'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_sd['Value'] = df_sd['trips']

df_sd = df_sd.rename(columns={ 
                              'wrk_nwrk_purp': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})
df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'B1.2'
df_sd['Metric_name'] = 'Commute trips on network'
df_sd['Submetric'] = 'B1.2.5'
df_sd['Description'] = 'Number of linked rail trips for commuting purpose(home-based work)'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Units'] = 'Trips'
df_sd['Total_Increment'] = ''

## B1.4 

In [22]:
non_work_trips = df_trn_rail.loc[df_trn_rail['wrk_nwrk_purp'] == 'non-work']

In [23]:
df_region_period = non_work_trips.groupby(['Mode', 'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_region_period['Value'] = df_region_period['trips']
df_region_period = df_region_period.rename(columns={'wrk_nwrk_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'B1.4'
df_region_period['Metric_name'] = 'Non-work trips on network'
df_region_period['Submetric'] = 'B1.4.1'
df_region_period['Description'] = 'number of linked rail trips for non-work purpose'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = 'Megaregion'
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'Trips'
df_region_period['Total_Increment'] = ''

In [24]:
df_tb = non_work_trips.groupby(['transbay_od', 'Mode', 'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_tb = df_tb.loc[df_tb['transbay_od'] == 1]

df_tb['Value'] = df_tb['trips']/df_tb['trips'].sum()
df_tb = df_tb.rename(columns={'wrk_nwrk_purp': 'Purpose', 'transbay_od' : 'Zone_ID'})
df_tb = df_tb[['Mode', 'Period', 'Purpose', 'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'B1.4'
df_tb['Metric_name'] = 'Non-work trips on network'
df_tb['Submetric'] = 'B1.4.2'
df_tb['Description'] = 'number of linked rail trips for non-work purpose'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
#df_region_period['Zone_ID'] =
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'Trips'
df_tb['Total_Increment'] = ''

In [25]:
df_cnty = non_work_trips.groupby(['orig_county', 'dest_county', 'Mode', 
                            'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_cnty['Value'] = df_cnty['trips']
df_cnty = df_cnty.rename(columns={
                                  'wrk_nwrk_purp': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'B1.4'
df_cnty['Metric_name'] = 'Non-work trips on network'
df_cnty['Submetric'] = 'B1.4.3'
df_cnty['Description'] = 'number of linked rail trips for non-work purpose'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Units'] = 'Trips'
df_cnty['Total_Increment'] = ''

In [26]:
df_rdm = non_work_trips.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
                           'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_rdm['Value'] = df_rdm['trips']
df_rdm = df_rdm.rename(columns={
                                'wrk_nwrk_purp': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'B1.4'
df_rdm['Metric_name'] = 'Non-work trips on network'
df_rdm['Submetric'] = 'B1.4.4'
df_rdm['Description'] = 'number of linked rail trips for non-work purpose'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Units'] = 'Trips'
df_rdm['Total_Increment'] = ''

In [27]:
df_sd = non_work_trips.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
                          'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_sd['Value'] = df_sd['trips']

df_sd = df_sd.rename(columns={ 
                              'wrk_nwrk_purp': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})
df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'B1.4'
df_sd['Metric_name'] = 'Non-work trips on network'
df_sd['Submetric'] = 'B1.4.5'
df_sd['Description'] = 'number of linked rail trips for non-work purpose'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Units'] = 'Trips'
df_sd['Total_Increment'] = ''

## C2.2

In [1]:
df_business = df_trn_rail.loc[df_trn_rail['link21_trip_purp'] == 'business']

NameError: name 'df_trn_rail' is not defined

In [ ]:
df_region_period = df_business.groupby(['Mode', 'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_region_period['Value'] = df_region_period['trips']
df_region_period = df_region_period.rename(columns={'wrk_nwrk_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'C2.2'
df_region_period['Metric_name'] = 'Business to Business trips'
df_region_period['Submetric'] = 'C2.2.1'
df_region_period['Description'] = 'number of linked rail trips for non-work purpose'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = 'Megaregion'
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'Trips'
df_region_period['Total_Increment'] = ''

In [ ]:
df_tb = df_business.groupby(['transbay_od', 'Mode', 'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_tb = df_tb.loc[df_tb['transbay_od'] == 1]

df_tb['Value'] = df_tb['trips']/df_tb['trips'].sum()
df_tb = df_tb.rename(columns={'wrk_nwrk_purp': 'Purpose', 'transbay_od' : 'Zone_ID'})
df_tb = df_tb[['Mode', 'Period', 'Purpose', 'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'C2.2'
df_tb['Metric_name'] = 'Business to Business trips'
df_tb['Submetric'] = 'C2.2.2'
df_tb['Description'] = 'number of linked rail trips for business purpose'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
#df_region_period['Zone_ID'] =
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'Trips'
df_tb['Total_Increment'] = ''

In [ ]:
df_cnty = df_business.groupby(['orig_county', 'dest_county', 'Mode', 
                            'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_cnty['Value'] = df_cnty['trips']
df_cnty = df_cnty.rename(columns={
                                  'wrk_nwrk_purp': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'C2.2'
df_cnty['Metric_name'] = 'Business to Business trips'
df_cnty['Submetric'] = 'C2.2.3'
df_cnty['Description'] = 'number of linked rail trips for business purpose'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Units'] = 'Trips'
df_cnty['Total_Increment'] = ''

In [ ]:
df_rdm = df_business.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
                           'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_rdm['Value'] = df_rdm['trips']
df_rdm = df_rdm.rename(columns={
                                'wrk_nwrk_purp': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'C2.2'
df_rdm['Metric_name'] = 'Business to Business trips'
df_rdm['Submetric'] = 'C2.2.4'
df_rdm['Description'] = 'number of linked rail trips for business purpose'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Units'] = 'Trips'
df_rdm['Total_Increment'] = ''

In [ ]:
df_sd = non_work_trips.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
                          'Period', 'wrk_nwrk_purp'])['trips'].sum().reset_index()

df_sd['Value'] = df_sd['trips']

df_sd = df_sd.rename(columns={ 
                              'wrk_nwrk_purp': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})
df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'C2.2'
df_sd['Metric_name'] = 'Business to Business tripsk'
df_sd['Submetric'] = 'C2.2.5'
df_sd['Description'] = 'number of linked rail trips for business purpose'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Units'] = 'Trips'
df_sd['Total_Increment'] = ''